In [ ]:
import cv2
import numpy as np
import os
import pyttsx3

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input

In [ ]:
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.models import Sequential
from keras import layers

In [ ]:
import cv2
import numpy as np

In [ ]:
from keras.models import load_model
loaded_model = load_model('F:\\temp\\models\\model_9classes_v6.h5')

In [ ]:
class skinDetector(object):
    #class constructor
    def __init__(self, img):
        self.image = img
#         self.image_path=img
        if self.image is None:
            print("IMAGE NOT FOUND")
            exit(1)                          
        #self.image = cv2.resize(self.image,(600,600),cv2.INTER_AREA)	
        self.HSV_image = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        self.YCbCr_image = cv2.cvtColor(self.image, cv2.COLOR_BGR2YCR_CB)
        self.binary_mask_image = self.HSV_image
#================================================================================================================================
    #function to process the image and segment the skin using the HSV and YCbCr colorspaces, followed by the Watershed algorithm
    def find_skin(self):
        self.__color_segmentation()
        self.__region_based_segmentation()

#================================================================================================================================
    #Apply a threshold to an HSV and YCbCr images, the used values were based on current research papers along with some
    # empirical tests and visual evaluation
    def __color_segmentation(self):
        lower_HSV_values = np.array([0, 40, 0], dtype = "uint8")
        upper_HSV_values = np.array([25, 255, 255], dtype = "uint8")

        lower_YCbCr_values = np.array((0, 138, 67), dtype = "uint8")
        upper_YCbCr_values = np.array((255, 173, 133), dtype = "uint8")

        #A binary mask is returned. White pixels (255) represent pixels that fall into the upper/lower.
        mask_YCbCr = cv2.inRange(self.YCbCr_image, lower_YCbCr_values, upper_YCbCr_values)
        mask_HSV = cv2.inRange(self.HSV_image, lower_HSV_values, upper_HSV_values) 

        self.binary_mask_image = cv2.add(mask_HSV,mask_YCbCr)

#================================================================================================================================
    #Function that applies Watershed and morphological operations on the thresholded image
    def __region_based_segmentation(self):
        #morphological operations
        image_foreground = cv2.erode(self.binary_mask_image,None,iterations = 3)     	#remove noise
        dilated_binary_image = cv2.dilate(self.binary_mask_image,None,iterations = 3)   #The background region is reduced a little because of the dilate operation
        ret,image_background = cv2.threshold(dilated_binary_image,1,128,cv2.THRESH_BINARY)  #set all background regions to 128

        image_marker = cv2.add(image_foreground,image_background)   #add both foreground and backgroud, forming markers. The markers are "seeds" of the future image regions.
        image_marker32 = np.int32(image_marker) #convert to 32SC1 format

        cv2.watershed(self.image,image_marker32)
        m = cv2.convertScaleAbs(image_marker32) #convert back to uint8 

        #bitwise of the mask with the input image
        ret,image_mask = cv2.threshold(m,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        output = cv2.bitwise_and(self.image,self.image,mask = image_mask)
        cv2.imwrite("F:\\temp\\output.jpg",output)
        #show the images
#         self.show_image(self.image)
#         self.show_image(image_mask)
#         self.show_image(output)


#================================================================================================================================
    def show_image(self, image):
        cv2.imshow("Image",image)
        cv2.waitKey(0)
        cv2.destroyWindow("Image")

In [ ]:
# x=cv2.imread('F:\\temp\\test_videos\\photos\\test1\\frame10.jpg')
# detector = skinDetector(x)
# detector.find_skin()

In [ ]:
train_data_augmentor = ImageDataGenerator(samplewise_center=True, 
                                    samplewise_std_normalization=True)

In [ ]:
vidcap = cv2.VideoCapture('F:\\temp\\test_videos\\test1.mp4')
success,image = vidcap.read()
count = 0
text=""
while success:
#     cv2.imwrite("F:\\temp\\test_videos\\photos\\test2\\frame%d.jpg" % count, image)     # save frame as JPEG file
    detector = skinDetector(image)
    detector.find_skin()
    if(count%15==0):
#         mask = cv2.imread("F:\\temp\\output.jpg")
        x = cv2.imread("F:\\temp\\output.jpg").reshape(1,216,384,3)
        x = np.array(x, dtype=np.float64)
        x = train_data_augmentor.standardize(x)
        pred=loaded_model.predict(x)
        val=np.argmax(pred)
        if(val==0):
            str="a"
        if(val==1):
            str="d"
        if(val==2):
            str="f"
        if(val==3):
            str="g"
        if(val==4):
            str="h"
        if(val==5):
            str="i"
        if(val==6):
            str="k"
        if(val==7):
            str="l"
        if(val==8):
            str="o"
        if(text!=""):
            last=text[-1]
            if(str!=last):
                text=text+" "+str
        else:
            text=text+str
    mask = cv2.imread("F:\\temp\\output.jpg")
    cv2.putText(image, str, (280,50), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0),4)
    cv2.imshow("Image2", mask)
    cv2.imshow("Image",image)
    success,image = vidcap.read()
#   print('Read a new frame: ', success)
    count += 1
    key_pressed = cv2.waitKey(1) & 0xFF
    if(key_pressed==ord('a')):
        break
vidcap.release()
cv2.destroyAllWindows()

In [ ]:
train_data_augmentor = ImageDataGenerator(samplewise_center=True, 
                                    samplewise_std_normalization=True)

In [ ]:
a="dfdsfgf"

In [ ]:
a[-1]

In [ ]:
x = cv2.imread("F:\\temp\\output.jpg").reshape(1,216,384,3)

In [ ]:
x = np.array(x, dtype=np.float64)

In [ ]:
x = train_data_augmentor.standardize(x)

In [ ]:
pred=loaded_model.predict(x)

In [ ]:
pred

In [ ]:
engine = pyttsx3.init()  
engine.say(text)
engine.runAndWait() 

In [ ]:
vidcap = cv2.VideoCapture('F:\\temp\\test_videos\\test1.mp4')
success,image = vidcap.read()
count = 0
while success:
    print(type(image))
#     cv2.imshow("Image",image)
    success,image = vidcap.read()
    if(count==10): x=image
#     cv2.imshow("Image",image)
#   print('Read a new frame: ', success)
    count += 1
# vidcap.release()
# cv2.destroyAllWindows()

In [ ]:
cv2.imshow("Image", x)

In [ ]:
cv2.imwrite("F:\\temp\\output.jpg",x)

In [ ]:
x=cv2.imread("F:\\temp\\output.jpg")

In [ ]:
cv2.imshow("Image", x)
cv2.waitKey(0)
cv2.destroyWindow("Image")